In [1]:
from azureml.core import Workspace, Experiment

# name = "<Workspace Name>"
# subid = "<Subscription ID>"
# rg = "<Resource Group>"

name = "WorkSpaceOne"
subid = "e8e8da26-0fa0-48b5-a094-ad8115fd47b3"
rg = "UdacityAMLND"

ws = Workspace.get(name=name,subscription_id=subid,resource_group=rg)
exp = Experiment(workspace=ws, name="udacity-project-1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: WorkSpaceOne
Azure region: eastus2
Subscription id: e8e8da26-0fa0-48b5-a094-ad8115fd47b3
Resource group: UdacityAMLND


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_cluster = ComputeTarget(ws,"ml-cluster")
except:
    compute_cluster_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",min_nodes=0,max_nodes=4,idle_seconds_before_scaledown=240)
    compute_cluster = ComputeTarget.create(ws,"ml-cluster",compute_cluster_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C':uniform(0.05,30),
    '--max_iter':choice(40,60,80,100,120)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=4,slack_factor=0.2,)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=".",
    compute_target=compute_cluster,
    entry_script="train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,primary_metric_name="Accuracy",primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,max_concurrent_runs=4,policy=policy,estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.widgets import RunDetails

hd_submit = exp.submit(config=hyperdrive_config)

RunDetails(hd_submit).show()

hd_submit.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3a519658-1bf1-46a4-b35b-80c611b7c621
Web View: https://ml.azure.com/runs/HD_3a519658-1bf1-46a4-b35b-80c611b7c621?wsid=/subscriptions/e8e8da26-0fa0-48b5-a094-ad8115fd47b3/resourcegroups/UdacityAMLND/workspaces/WorkSpaceOne&tid=8a794c71-bef8-4d45-8cb2-b0c18444be8b

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-12-27T08:11:34.047279][API][INFO]Experiment created<END>\n""<START>[2021-12-27T08:11:34.694627][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-12-27T08:11:35.195897][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_3a519658-1bf1-46a4-b35b-80c611b7c621
Web View: https://ml.azure.com/runs/HD_3a519658-1bf1-46a4-b35b-80c611b7c621?wsid=/subscriptions/e8e8da26-0fa0-48b5-a094-ad8115fd47b3/resourcegroups/UdacityAMLND/workspaces/WorkSpaceOne&tid=8a794c71-bef8-4d45-8cb2-b0c18444be8b



{'runId': 'HD_3a519658-1bf1-46a4-b35b-80c611b7c621',
 'target': 'ml-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-12-27T08:11:33.789688Z',
 'endTimeUtc': '2021-12-27T08:32:12.612056Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1e8d3730-5471-4170-91bc-3edc7dd1666c',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': 'infinite_space_size',
  'score': '0.9113808801213961',
  'best_child_run_id': 'HD_3a519658-1bf1-46a4-b35b-80c611b7c621_18',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://workspaceone4929537462.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_3a519658-1bf1-46a4-

In [8]:
import joblib
# Get your best run and save the model from that run.
best_run_hd = hd_submit.get_best_run_by_primary_metric()
best_model_hd = best_run_hd.register_model(model_name = 'udacityprojecthyperdrive',model_path='.')

In [9]:
best_model_hd

Model(workspace=Workspace.create(name='WorkSpaceOne', subscription_id='e8e8da26-0fa0-48b5-a094-ad8115fd47b3', resource_group='UdacityAMLND'), name=udacityprojecthyperdrive, id=udacityprojecthyperdrive:1, version=1, tags={}, properties={})

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
tab_data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=tab_data_url)

In [11]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

df = pd.concat([x,y],axis=1)
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [12]:
def_datastore = ws.get_default_datastore()
print(def_datastore)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-dd229ad6-654c-412f-bad5-dbea6cad51a2",
  "account_name": "workspaceone4929537462",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [13]:
from azureml.data.datapath import DataPath
data_final = TabularDatasetFactory.register_pandas_dataframe(dataframe = df,
                                                       target=DataPath(datastore=def_datastore,path_on_datastore='./automl/data'),
                                                       name='bankmarketing')

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to ./automl/data/7fa8e287-56ca-4c8a-8d04-d64c2026bbd0/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [14]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_final,
    label_column_name='y',
    n_cross_validations=8,
    compute_target=compute_cluster,
    max_cores_per_iteration=-1,
    max_concurrent_iterations = 4,
    enable_early_stopping = True
    )

In [15]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config,show_output=True)

Submitting remote run.
No run_configuration provided, running on ml-cluster with default configuration
Running on remote compute: ml-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-1,AutoML_e83c9934-4b3c-4ce8-a4b4-b865b45356d9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Generating features for the dataset.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-----------------------

In [16]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run = automl_run.get_best_child()

automl_model = automl_best_run.register_model(model_name = 'automl_best_model',model_path='.')


In [17]:
automl_model

Model(workspace=Workspace.create(name='WorkSpaceOne', subscription_id='e8e8da26-0fa0-48b5-a094-ad8115fd47b3', resource_group='UdacityAMLND'), name=automl_best_model, id=automl_best_model:1, version=1, tags={}, properties={})

In [18]:
# Delete Cluster
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

